# Laboratório 07: Avaliação de Sistemas de RI

Antes de responder as perguntas deste laboratório, precisamos importar as extensões que usaremos no decorrer da atividade, bem como criar os índices e implementar os algoritmos binário, TF, BM25 e TF-IDF da última atividade.


*   **Importação das extensões necessárias** 

In [0]:
import csv
import pandas as pd
import numpy as np
import nltk
import re
import collections
import bisect
from nltk.tokenize import RegexpTokenizer
nltk.download('stopwords')

*   **Leitura do CSV** 

In [0]:
colecao = pd.read_csv('https://raw.githubusercontent.com/LDVictor/ri_lab_07/master/results.csv')
documentos = colecao['text']

*   **Criação dos índices** 

In [0]:
tokenizador = RegexpTokenizer(r'([A-Za-zÁáÉéÍíÓóÚúÃãÕõÇçÂâÊê]{3,27})')
stopwords = nltk.corpus.stopwords.words('portuguese') 
indices = {}
n = 0

for texto in documentos:
  palavras = [palavra for palavra in tokenizador.tokenize(texto.lower())
           if not bool(re.search(r'\d', palavra))
           and palavra not in stopwords and len(palavra) >= 3]  
  n += 1
  for t in palavras:
    if t not in indices.keys():
      indices[t] = []
    indices[t].append(n)
    
for elemento in indices.items():
  d = dict(collections.Counter(elemento[1]))
  indices[elemento[0]] = list(d.items())

*   **Implementação dos algoritmos** 

In [0]:
def modeloVetorialBinario(consulta, documento):
  pontos = 0
  tokens_documento = documento.split()
  tokens_consulta = consulta.split()
  
  for token in tokens_consulta:
    pontos += (token in tokens_documento)
 
  return pontos

def modeloVetorialTF(consulta, documento):
  pontos = 0
  tokens_documento = documento.split()
  tokens_consulta = consulta.split()
  
  for token in tokens_consulta:
    pontos += tokens_documento.count(token)
  
  return pontos

def modeloVetorialTFIDF(consulta, documento):
  pontos = 0
  tokens_documento = documento.split()
  tokens_consulta = consulta.split()
  
  for token in tokens_consulta:
    cwd = tokens_documento.count(token)
    pontos += cwd * indices[token][-1]
  
  return round(pontos, 2)

def modeloVetorialBM25(consulta, documento, n):
  pontos = 0
  tokens_documento = documento.split()
  tokens_consulta = consulta.split()
  
  palavras = [palavra for palavra in tokens_consulta if palavra in tokens_documento]
    
  for palavra in palavras:
    cwd = tokens_documento.count(palavra)
    dfw = len(indices[palavra][:-1])
    pontos += (((k+1) * cwd) / (cwd + k)) * np.log10(((m+1) / dfw))
  
  return round(pontos, 2)

# Questão 1

**Escolha um documento dentre aqueles da base do aluno Bernardi e crie uma consulta que você acha que tem boas chances de recuperar este documento.**

In [0]:
...

**Em seguida, avalie os resultados de tal consulta usando a métrica de avaliação Reciprocal Rank.**

In [0]:
...

((Explicação dos resultados))

# Questão 2

**A partir do gabarito fornecido em OBS1, calcule o MAP para cada algoritmo abaixo e aponte qual obteve o melhor resultado. Para os cálculos do MAP, considere que um documento é relevante para uma dada consulta se este documento estiver entre os documentos do gabarito para essa consulta, senão ele deve ser considerado irrelevante.**

Inicialmente, iremos importar o gabarito fornecido no enunciado da atividade.

In [0]:
colecao_obs1 = pd.read_csv('https://raw.githubusercontent.com/LDVictor/ri_lab_07/master/results_final.json')

Agora, podemos calcular o MAP para cada um dos quatro algoritmos a partir da nova coleção.

In [0]:
...

((Explicação do melhor resultado))

# Questão 3

**Repita Q2 usando a avaliação multi-nível DCG. Utilize o campo "level" do gabarito para o cálculo do DCG e do idealDCG. Use uma janela de 5 documentos.**

In [0]:
...

((Explicação do melhor resultado))